# Searching Algorithm
- categories: [Python, SearchingAlgorithm]

In [1]:
# if we simply use a very large number (1,000,000 for example), we don't need any library
import math

In [2]:
# define a game class to return state and all actions at that state
class CountGame:
    def __init__(self, goal):
        self.goal = goal
        
    def state(self):
        return self.goal
    
    def is_end(self, state):
        return state == self.goal
    
    def succ_and_cost(self, state):
        output = []
        if state <= self.goal // 2:
            output.append(('tram', state * 2, 2))
        if state < self.goal:
            output.append(('walk', state + 1, 1))
        return output

In [3]:
# recursion method - brute force
def recursion(problem):
    result = {
        'cost': math.inf,
        'history': []
    }
    def recurse(state, history, total_cost):
        if problem.is_end(state):    # return if reach end state
            if total_cost < result['cost']:    # update best result if total cost is lower
                result['cost'] = total_cost
                result['history'] = history
            return
        for action, new_state, cost in game.succ_and_cost(state):
            next_history = history + [(action, new_state, total_cost + cost)]
            recurse(new_state, next_history, total_cost + cost)
    
    recurse(1, [], 0)
    return result

In [4]:
%%time
game = CountGame(200)
recursion(game)

CPU times: user 6.5 s, sys: 10.9 ms, total: 6.51 s
Wall time: 6.52 s


{'cost': 15,
 'history': [('walk', 2, 1),
  ('walk', 3, 2),
  ('tram', 6, 4),
  ('tram', 12, 6),
  ('tram', 24, 8),
  ('walk', 25, 9),
  ('tram', 50, 11),
  ('tram', 100, 13),
  ('tram', 200, 15)]}

### takes more than 6 seconds for 200! Can we do better?

In [5]:
# introducing "future_cost" that depend on state only
def dynamic_programming(problem):
    cashe = {}    # store calculated items in cashe to save computation time
    result = {
        'cost': math.inf,
        'history': []
    }
    def future_cost(state):
        if cashe.get(state):
            return cashe[state]
        if problem.is_end(state):
            return 0, []
        # minor change to trace action history and store in cashe
        tot_cost = math.inf
        for action, new_state, cost in problem.succ_and_cost(state):
            f_cost, new_history = future_cost(new_state)
            if cost + f_cost < tot_cost:
                tot_cost = cost + f_cost
                next_action = [(action, new_state, tot_cost)] + new_history
        cashe[state] = tot_cost, next_action
        # original code in lecutre video, action history is not output
        # cashe[state] = min(cost + future_cost(new_state) \
        #                    for action, new_state, cost in problem.succ_and_cost(state))
        return cashe[state]
    result['cost'], result['history'] = future_cost(1)
    return result

In [6]:
%%time
game = CountGame(1000)
dynamic_programming(game)

CPU times: user 4.78 ms, sys: 756 µs, total: 5.54 ms
Wall time: 5.51 ms


{'cost': 22,
 'history': [('walk', 2, 22),
  ('walk', 3, 21),
  ('tram', 6, 20),
  ('walk', 7, 18),
  ('tram', 14, 17),
  ('walk', 15, 15),
  ('tram', 30, 14),
  ('walk', 31, 12),
  ('tram', 62, 11),
  ('tram', 124, 9),
  ('walk', 125, 7),
  ('tram', 250, 6),
  ('tram', 500, 4),
  ('tram', 1000, 2)]}

### and now it takes less than 9 ms for 1000